## Automated Document Processing Pipeline: Using Foundation Models for Smart Documents Chunking and Knowledge Base Integration


##### This Notebook requires an existing Knowledge base on bedrock. To create a knowledge base you can execute the code provided in :
https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb
### Challenge


Chunking is the process of dividing documents into smaller sections, or "chunks," before embedding them into a Knowledge Base. This process enhances retrieval efficiency and precision. There are several chunking strategies available, each suited to different types of content and document structures. Examples of chunking strategies supported by Amazon Bedrock are: 
- FIXED_SIZE: Splitting documents into chunks of the approximate size that set.
- HIERARCHICAL: Splitting documents into layers of chunks where the first layer contains large chunks, and the second layer contains smaller chunks derived from the first layer.
- SEMANTIC: Split documents into chunks based on groups of similar content derived with natural language processing.

FIXED_SIZE is useful in scenarios requiring predictable chunk sizes for processing. HIERARCHICAL chunking is appropriate when dealing with complex, nested data structures. Whereas Semantic Chunking is useful when dealing with complex, contextual information and processing documents where meaning across sentences is highly interconnected. 
The main drawbacks of semantic chunking include higher computational requirements, limited effectiveness across different languages and scalability challenges with large datasets. The main drawbacks of hierarchical chunking include higher computational overhead, difficulty in managing deep hierarchies and slower query performance at deeper levels.

Selecting the right chunking strategy require understanding of benefits and limitations of each strategy in the context of analyzed documents, business requirements and SLAs. To determine the adequate chunking strategy, developer needs to manually assess document before selecting a strategy. The final choice is a balance between efficiency, accuracy, and practical constraints of the specific use case


### Approach presented in this notebook

The approach presented in this notebook leverages Foundation Models (FMs) to automate document analysis and ingestion into an Amazon Bedrock Knowledge Base, replacing manual human assessment. The system automatically:
- Analyzes document structure and content
- Determines the optimal chunking strategy for each document
- Generates appropriate chunking configurations
- Executes the document ingestion process

The solution recognizes that different documents require different chunking approaches, and therefore performs individual assessments to optimize content segmentation for each document type. This automation streamlines the process of building and maintaining knowledge bases while ensuring optimal document processing for better retrieval and usage.

The key idea in this work is using FMs to intelligently analyze and process documents, rather than relying on predetermined or manual chunking strategies.

### Notebook Walkthrough
The pipeline streamlines the entire process from document analysis to knowledge base population, making it efficient to prepare documents for advanced query and retrieval operations.

![data_ingestion](./img/chunkingAdvs.jpg)
### Steps: 

1. Create Amazon Bedrock Knowledge Base execution role and S3 bucket used as data sources and configure necessary IAM policies 
2. Process files within target folder. For each document, analyze and recommends an optimal chunking strategy (FIXED_SIZE/NONE/HIERARCHICAL/SEMANTIC) and specific configuration parameters
3. Upload analyzed files to designated S3 buckets and configure buckets as data source for Bedrock KB
4. Initiate ingestion job 
5. Verify data accessibility and accuracy


### Setup

In [ ]:
%pip install --force-reinstall -q -r ./requirements.txt --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Initiate parameters 

##### Knowledge base ID should have been created from first notebook (https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb) or similar
- To get knowledge Base Id using Bedrock console, look int Amazon Bedrock > knowledgebase> knowledgebase </b>


In [ ]:
import boto3
import json

# create a boto3 session to dynamically get and set the region name
session = boto3.Session()

AWS_REGION = session.region_name
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
bedrock_agent_client = session.client("bedrock-agent", region_name=AWS_REGION)
# model was run in us-west-2 , if you are using us-east-1 then change model ID to "us.anthropic.claude-3-5-sonnet-20241022-v2:0" #
MODEL_NAME = "anthropic.claude-3-5-sonnet-20241022-v2:0"
datasources = []

# create a folder data if not yet done and
path = "data"

kb_id = "xxxx"  # Retrieve KB First  # update value here with your KB ID
kb = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
# get bedrock_kb_execution ID - this role should have been create from notebook creating KB
bedrock_kb_execution_role_arn = kb["knowledgeBase"]["roleArn"]
bedrock_kb_execution_role = bedrock_kb_execution_role_arn.split("/")[-1]
account_id = boto3.client("sts").get_caller_identity()["Account"]
print(bedrock_kb_execution_role)

### Supporting functions
##### Function 1 -  Createbucket: Checks if an S3 bucket exists and creates it if it doesn't. 
##### Function 2 - Upload_file: Upload_files to bucket: Upload a file to an S3 bucket
##### Function 3 - List all files in a specified directory
##### Function 4 - Delete a S3 bucket and all objects included within

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def createbucket(bucketname):
    """
    Checks if an S3 bucket exists and creates it if it doesn't.
    """
    try:
        s3_client = boto3.client("s3")
        s3_client.head_bucket(Bucket=bucketname)
        print(f"Bucket {bucketname} Exists")
    except ClientError as e:
        print(f"Creating bucket {bucketname}")
        if AWS_REGION == "us-east-1":
            s3bucket = s3_client.create_bucket(Bucket=bucketname)
        else:
            s3bucket = s3_client.create_bucket(
                Bucket=bucketname,
                CreateBucketConfiguration={"LocationConstraint": AWS_REGION},
            )


def upload_file(file_name, bucket, object_name=None):
    """
    Upload a file to an S3 bucket
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client("s3")
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True


def listfile(folder):
    """
    List all files in a specified directory.
    """
    dir_list = os.listdir(folder)
    return dir_list


def delete_bucket_and_objects(bucket_name):
    """
    Delete a S3 bucket and all objects included in
    """
    # Create an S3 client
    s3_client = boto3.client("s3")
    # Create an S3 resource
    s3 = boto3.resource("s3")
    bucket = s3.Bucket(bucket_name)
    bucket.objects.all().delete()
    # Delete the bucket itself
    bucket.delete()

### Standard prompt completion function

In [ ]:
def get_completion(prompt):
    body = json.dumps(
        {
            "anthropic_version": "",
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": "",
        }
    )
    response = bedrock.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get("body").read())
    return response_body.get("content")[0].get("text")

###  Download and prepare datasets 
The test dataset consists of two documents, these files will serve as test cases to validate the model's ability to correctly identify and recommend the most appropriate chunking strategy for each document type.

In [ ]:
# if not yet created, create folder already
#!mkdir -p ./data

from urllib.request import urlretrieve

urls = [
    "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf",
    "https://www.apple.com/newsroom/pdfs/Q3FY18ConsolidatedFinancialStatements.pdf",
]
filenames = [
    "AMZN-2022-Shareholder-Letter.pdf",
    "Q3FY18ConsolidatedFinancialStatements.pdf",
]
data_root = "./data/"
for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

### Create 3 S3 buckets, one per chunking strategy
##### Important Note on AWS Bedrock Knowledge Base Configuration:

The chunking strategy for a data source is permanent and cannot be modified after initial configuration. To address this challenge, we are implementing the following structure:

Three separate S3 buckets will be created, each dedicated to a specific chunking strategy:
- Bucket for semantic chunking
- Bucket for hierarchical chunking
- Bucket for hybrid chunking

These separate buckets approach allows us to maintain different chunking strategies for different document types within the same knowledge base system, ensuring optimal processing for each document category.


In [ ]:
import random

suffix = random.randrange(200, 900)
s3_client = boto3.client("s3")
bucket_name_semantic = "kb-dataset-bucket-semantic-" + str(suffix)
bucket_name_fixed = "kb-dataset-bucket-fixed-" + str(suffix)
bucket_name_hierachical = "kb-dataset-bucket-hierarchical-" + str(suffix)
s3_policy_name = "AmazonBedrockS3PolicyForKnowledgeBase_" + str(suffix)
createbucket(bucket_name_semantic)
createbucket(bucket_name_fixed)
createbucket(bucket_name_hierachical)

### Create S3 policies and attach to existing Bedrock role


In [ ]:
account_number = boto3.client("sts").get_caller_identity().get("Account")
iam_client = session.client("iam")
iam_client = session.client("iam")
s3_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": ["s3:GetObject", "s3:ListBucket"],
            "Resource": [
                f"arn:aws:s3:::{bucket_name_semantic}",
                f"arn:aws:s3:::{bucket_name_semantic}/*",
                f"arn:aws:s3:::{bucket_name_fixed}",
                f"arn:aws:s3:::{bucket_name_fixed}/*",
                f"arn:aws:s3:::{bucket_name_hierachical}",
                f"arn:aws:s3:::{bucket_name_hierachical}/*",
            ],
            "Condition": {"StringEquals": {"aws:ResourceAccount": f"{account_number}"}},
        }
    ],
}
s3_policy = iam_client.create_policy(
    PolicyName=s3_policy_name,
    PolicyDocument=json.dumps(s3_policy_document),
    Description="Policy for reading documents from s3",
)

# fetch arn of this policy
s3_policy_arn = s3_policy["Policy"]["Arn"]
iam_client = session.client("iam")
fm_policy_arn = f"arn:aws:iam::{account_number}:policy/{s3_policy_name}"
iam_client.attach_role_policy(
    RoleName=bedrock_kb_execution_role, PolicyArn=fm_policy_arn
)

### Document Analysis

Purpose: analyzes PDF documents using an LLM to recommend the optimal chunking strategy and its associated parameters.
Input: PDF document
Output: The function recommends one of the following chunking strategies with specific parameters:
- HIERARCHICAL Chunking:
    - Maximum parent chunk token size
    - Maximum child chunk token size
    - Overlap tokens
    - Rationale for recommendation
- SEMANTIC Chunking:
    - Maximum tokens
    - Buffer size
    - Breakpoint percentile threshold
    - Rationale for recommendation
- FIXED-SIZE Chunking:
    - Maximum tokens
    - Overlap percentage
    - Rationale for recommendation


In [ ]:
def chunking_advise(file):
    from langchain.document_loaders import PyPDFLoader

    my_docs = []
    my_strategies = []
    strategy = ""
    strategytext = ""
    path = "data"
    strategylist = []
    metadata = [dict(year=2023, source=file)]
    print("I am now analyzing the file:", file)
    file = path + "/" + file
    loader = PyPDFLoader(file)
    document = loader.load()
    loader = PyPDFLoader(file)
    document = loader.load()
    # print (document)
    prompt = f"""SYSTEM you are an advisor expert in LLM chunking strategies,
        USER can you analyze the type, content, format, structure and size of {document}. 
            1. See the actual document content
            2. Analyze its structure
            3. Examine the text format
            4. Understand the document length
            5. Review any hierarchical elements and Assess the semantic relationships within the content
            6. Evaluate the formatting and section breaks
        then advise on best LLM chunking Strategy based on this analysis. Recommend only one Strategy, however show recommended strategy preference ratio 
        Available strategies to recommend from are: FIXED_SIZE or NONE or HIERARCHICAL or SEMANTIC
        Decide on recommendation first and then, what is the recommendation?  """

    res = get_completion(prompt)
    print("my recommendation is:", res)
    return res


def chunking_configuration(strategy, file):

    prompt = f""" USER based on recommendation provide in {strategy} , provide for {file} a recommended chunking configuration, 
        if you recommend HIERARCHICAL chunking then provide recommendation for: 
        Parent: Maximum parent chunk token size. 
        Child: Maximum child chunk token size and Overlap Tokens: Number of overlap tokens between each parent chunk and between each parent and its children.
        If recommendation is HIERARCHICAL then provide response using JSON format
        with the keys as \”Recommend only one Strategy\”, \”Maximum Parent chunk token size\”, \”Maximum child chunk token size\”,\”Overlap Tokens\”,
        \"Rational: please explain rational for decision and explain why each other choice is not preferred, keep rational to 100 words maximum. \” . 
        provide crisp and clear answer, 
        if you recommend SEMANTIC then provide response using  JSON format with
        the keys as \”Recommend only one Strategy\”,\”Maximum tokens\”, \”Buffer size\”,\”Breakpoint percentile threshold\”, 
        Buffer size should be less or equal than 1 , Breakpoint percentile threshold should >= 50
        \”Rational: please explain rational for decision and explain why each other choice is not preferred, keep rational to 100 words maximum. \” . 
        provide crisp and clear answer, 
        do not provide recommendation if not enough data inputs and say sorry I need more data,
        if you recommend FIXED_SIZE then provide response using  JSON format with
        the keys as \”Recommend only one Strategy\”,\”maxTokens\”, \”overlapPercentage \”,
        \”Rational: please explain rational for decision and explain why each other choice is not preferred, keep rational to 100 words maximum. \” .
        provide crisp and clear answer, 
        do not provide recommendation if not enough data inputs and say sorry I need more data"""

    res = get_completion(prompt)
    print(res)
    parsed_data = json.loads(res)
    return parsed_data

### Ingest Documents By Strategy
Purpose: Configures AWS Bedrock Knowledge Base ingestion settings based on the recommended chunking strategy analysis.
- Interprets the recommended strategy from parsed_data
- Applies corresponding parameters to create appropriate configuration
- Selects the matching S3 bucket for the strategy
- Generates knowledge base metadata
- Returns all necessary components for Bedrock KB ingestion


In [ ]:
def ingestbystrategy(parsed_data):

    chunkingStrategyConfiguration = {}
    strategy = parsed_data.get("Recommend only one Strategy")

    # HIERARCHICAL Chunking
    if strategy == "HIERARCHICAL":
        p1 = parsed_data["Maximum Parent chunk token size"]
        p2 = parsed_data["Maximum child chunk token size"]
        p3 = parsed_data["Overlap Tokens"]
        bucket_name = bucket_name_hierachical
        name = f"bedrock-sample-knowledge-base-HIERARCHICAL"
        description = "Bedrock Knowledge Bases for and S3 HIERARCHICAL"
        chunkingStrategyConfiguration = {
            "chunkingStrategy": "HIERARCHICAL",
            "hierarchicalChunkingConfiguration": {
                "levelConfigurations": [{"maxTokens": p1}, {"maxTokens": p2}],
                "overlapTokens": p3,
            },
        }

    # SEMANTIC Chunking
    if strategy == "SEMANTIC":
        p3 = parsed_data["Maximum tokens"]
        p2 = int(parsed_data["Buffer size"])
        p1 = parsed_data["Breakpoint percentile threshold"]
        bucket_name = bucket_name_semantic
        name = f"bedrock-sample-knowledge-base-SEMANTIC"
        description = "Bedrock Knowledge Bases for and S3 SEMANTIC"
        chunkingStrategyConfiguration = {
            "chunkingStrategy": "SEMANTIC",
            "semanticChunkingConfiguration": {
                "breakpointPercentileThreshold": p1,
                "bufferSize": p2,
                "maxTokens": p3,
            },
        }
    # FIXED_SIZE Chunking
    if strategy == "FIXED_SIZE":
        p2 = int(parsed_data["overlapPercentage"])
        p1 = int(parsed_data["maxTokens"])
        bucket_name = bucket_name_fixed
        name = f"bedrock-sample-knowledge-base-FIXED"
        description = "Bedrock Knowledge Bases for and S3 FIXED"

        chunkingStrategyConfiguration = {
            "chunkingStrategy": "FIXED_SIZE",
            "semanticChunkingConfiguration": {"maxTokens": p1, "overlapPercentage": p2},
        }

    s3Configuration = {
        "bucketArn": f"arn:aws:s3:::{bucket_name}",
    }
    return (
        chunkingStrategyConfiguration,
        bucket_name,
        name,
        description,
        s3Configuration,
    )

### Create or retrieve data source from Amazon Bedrock Knowledge Base


In [ ]:
def createDS(
    name, description, knowledgeBaseId, s3Configuration, chunkingStrategyConfiguration
):
    response = bedrock_agent_client.list_data_sources(
        knowledgeBaseId=kb_id, maxResults=12
    )
    print(response)
    for i in range(len(response["dataSourceSummaries"])):
        print(response["dataSourceSummaries"][i]["name"], "::", name)
        print(response["dataSourceSummaries"][i]["dataSourceId"])
        if response["dataSourceSummaries"][i]["name"] == name:
            ds = bedrock_agent_client.get_data_source(
                knowledgeBaseId=knowledgeBaseId,
                dataSourceId=response["dataSourceSummaries"][i - 1]["dataSourceId"],
            )
            return ds
    ds = bedrock_agent_client.create_data_source(
        name=name,
        description=description,
        knowledgeBaseId=knowledgeBaseId,
        dataDeletionPolicy="DELETE",
        dataSourceConfiguration={
            # # For S3
            "type": "S3",
            "s3Configuration": s3Configuration,
        },
        vectorIngestionConfiguration={
            "chunkingConfiguration": chunkingStrategyConfiguration
        },
    )
    return ds

### Process PDF files by analyzing content, creating data sources, and uploading to S3.

#### Workflow:
1. Lists all files in specified directory
2. For each PDF:
    - Analyzes for optimal chunking strategy
    - Creates data source with recommended configuration
    - Uploads file to appropriate S3 bucket 

In [ ]:
s3_client = boto3.client("s3")
dir_list1 = listfile("data")
print(dir_list1)
strategylist = []
for file in dir_list1:
    if ".pdf" in file:
        chunkingStrategyConfiguration = []

        strategy = chunking_advise(file)
        strategy_conf = chunking_configuration(strategy, file)

chunkingStrategyConfiguration, bucket_name, name, description, s3Configuration = (
    ingestbystrategy(strategy_conf)
)
print("name", name)
datasources = createDS(
    name, description, kb_id, s3Configuration, chunkingStrategyConfiguration
)
with open(path + "/" + file, "rb") as f:
    s3_client.upload_fileobj(f, bucket_name, file)

### Ingestion jobs
##### please ensure that  Knowledge base role have the permission to  InvokeModel on resource: arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-text-v1

In [ ]:
from datetime import datetime
import time

"""
    Starts and monitors ingestion jobs for all data sources in a knowledge base.
"""
sources = bedrock_agent_client.list_data_sources(knowledgeBaseId=kb_id)
for i in range(len(sources["dataSourceSummaries"])):
    print("ds [dataSourceId]", sources["dataSourceSummaries"][i - 1]["dataSourceId"])
    start_job_response = bedrock_agent_client.start_ingestion_job(
        knowledgeBaseId=kb_id,
        dataSourceId=sources["dataSourceSummaries"][i - 1]["dataSourceId"],
    )
    job = start_job_response["ingestionJob"]
    print(job)
    # Get job
    while job["status"] != "COMPLETE":
        get_job_response = bedrock_agent_client.get_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=sources["dataSourceSummaries"][i - 1]["dataSourceId"],
            ingestionJobId=job["ingestionJobId"],
        )
        job = get_job_response["ingestionJob"]
    time.sleep(10)

### Try out KB and evaluate result score 
##### try both queries below

In [ ]:
# print  (response_ret )
def response_print(retrieve_resp):
    # structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num, chunk in enumerate(retrieve_resp["retrievalResults"], 1):
        print(f"Chunk -length : ", len(chunk["content"]["text"]), end="\n" * 2)
        print(f"Chunk {num} Location: ", chunk["location"], end="\n" * 2)
        print(f"Chunk {num} length: ", chunk["location"], end="\n" * 2)
        print(f"Chunk {num} Score: ", chunk["score"], end="\n" * 2)
        print(f"Chunk {num} Metadata: ", chunk["metadata"], end="\n" * 2)


query1 = "what is AWS annual revenue increase"

query2 = "what is iphone sales in 2018?"

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime")
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id,
    nextToken="string",
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults": 1,
        }
    },
    retrievalQuery={"text": query1},
)
print("Response shpould come from semantic chunked document:")
response_print(response_ret)

response_ret2 = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id,
    nextToken="string",
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults": 1,
        }
    },
    retrievalQuery={"text": query2},
)
print("Response shpould come from hierarchical chunked document:")
response_print(response_ret2)

##### Clean buckets 
#####  NOTE : please delete also Bedrock  KB if not required by other works and data sources 


In [ ]:
delete_bucket_and_objects(bucket_name_semantic)
delete_bucket_and_objects(bucket_name_fixed)
delete_bucket_and_objects(bucket_name_hierachical)

## Conclusion: 

This notebook presents a proof-of-concept approach that uses Foundation Models to automate chunking strategy selection for document processing. Please note:
- This is an experimental implementation
- Results should be validated before production use

This work serves as a starting point for automating chunking strategy decisions, but additional research and validation are needed to ensure reliability across diverse document types and use cases.

Suggested Next Steps:
- Expand testing across more document types
- Validate recommendations against human expert decisions
- Refine the model's decision-making criteria
- Gather performance metrics in real-world applications
- Build a validation Framework having a Ground Truth Database and including varied document types and structures using proven validation framework such as RAGA